In [2]:
from textblob.classifiers import NaiveBayesClassifier
import pandas as pd



import random

In [3]:
def normaliza_arquivos(data, sep):
    dataset=[]
    for line in data:
        values = line.split(sep)
        text = ','.join(values[0].split(',')[1:]).lower()
        label = values[1].lower()
        if len(text) > 1:
            if label == 'bom':
                label = 'b'
            elif label == 'ruim':
                label = 'r'
            elif label == 'neutro':
                label = 'n'
            text = text.lower().replace('"', '').replace("'", "")
            dataset.append((text, label))
    return dataset

# Lendo arquivo Fabiano

In [4]:
with open('treinamento/tweets_0_analise.csv', encoding='latin-1') as f:
    data0 = f.readlines()


# Lendo arquivo Carlos

In [5]:
with open('treinamento/tweets_1-PacoteDeMil.csv', encoding='utf-8') as f:
    data1 = f.readlines()


In [6]:
with open('treinamento/tweets_2.csv', encoding='utf-8') as f:
    data2 = f.readlines()

#LImpar arquivos

In [47]:
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

class WordProcessor():

    url = re.compile('https*://.*( |.|,|!)+')
    user_name = re.compile('@([0-9]|[A-z]|_| )+')
    hashtag = re.compile('#([A-z]|[0-9]| )+')

    def remove_url(self, text):
        return ''.join(self.url.split(text))

    def remove_user_mencions(self, text):
        return ''.join(self.user_name.split(text))
    
    def remove_hash_tag(self, text):
        return ''.join(self.hashtag.split(text))
    
    def remove_small_words(self, text, length):
        return ' '.join([word for word in text.split(' ') if len(word)>length])
    
    def remove_pontuation(self, text):
        return text.replace('!', '').replace('.', '').replace('?', '').replace(',', '')
                    
    
    def remove_stop_words(self, text):
        stop_words = stopwords.words('portuguese')
        text = text.lower()
        tokens = word_tokenize(text, language='portuguese')
        return ' '.join([ word for word in tokens if not word  in stop_words])


In [11]:
dataset = normaliza_arquivos(data0, ';')
dataset += normaliza_arquivos(data1, '\t')
dataset += normaliza_arquivos(data2, '\t')


# Limpando arquivo

No trecho abaixo sao removidas as mencoes a usuarios, as hashtags, url e as palavras de parada do portugues.

In [77]:
wp = WordProcessor()

clean_data = []
for sentence, classification in dataset:
    text_clean = wp.remove_url(sentence)
    text_clean = wp.remove_user_mencions(text_clean)
    text_clean = wp.remove_hash_tag(text_clean)
    text_clean = wp.remove_small_words(text_clean, 1)
    text_clean = wp.remove_stop_words(text_clean)
    text_clean = wp.remove_pontuation(text_clean)
    if len(text_clean) > 3:
        clean_data.append([text_clean, classification])

print(dataset[:10])
print(clean_data[:10])

[('se eu fosse mulher do justin ia torcer pros nossos filhos puxarem ao pai pq que bicha estranha eu sou mano', 'n'), ('acabei de lavar o cabelo', 'n'), ('mai tá falando ainda?', 'n'), ('#trocadepasses , @rogerflorestv , acho que o novo técnico do flamengo pode ser o wagner mancini..!!', 'n'), ('prefiro ser sincero com as pessoas, mesmo que algumas não gostem, antes sincero e verdadeiro do que falso.', 'b'), ('@ifvccabello sdv bb', 'n'), ('depende https://t.co/72u8z7yh1w', 'n'), ('deus, eu te amo.', 'b'), ('hj parei p refletir como uma pessoa faz falta, como sua vida fica avulsa sem a presença de quem vc sempre tem por perto, então valoriza', 'b'), ('já avisei hj minha mãe que esse mês eu vou furar a orelha', 'n')]
[['mulher justin ia torcer pros filhos puxarem pai pq bicha estranha mano', 'n'], ['acabei lavar cabelo', 'n'], ['mai tá falando ainda ', 'n'], ['acho novo técnico flamengo pode ser wagner mancini  ', 'n'], ['prefiro ser sincero pessoas  algumas gostem  antes sincero verdade

## Preparando teste

Separando massa em train e teste

In [89]:
import numpy as np


train_index = np.random.randint(len(clean_data), size=int(len(clean_data)*0.8), dtype=int)
train = [clean_data[i] for i in train_index]
test_index = np.random.randint(len(clean_data), size=int(len(clean_data)*0.2),dtype=int)
test = [clean_data[i] for i in test_index]

In [90]:
cl = NaiveBayesClassifier(train)

In [91]:
type(cl)

textblob.classifiers.NaiveBayesClassifier

In [92]:
result = 0
for i in test:
    text = i[0]
    res = cl.classify(text)
    if res == i[1]:
        result+=1


In [93]:
print(result)

370


In [94]:
print(len(test))

512


In [102]:
q=cl.prob_classify("isso eh triste")

In [103]:
q.max()

'r'

In [104]:
print(q.max())

r


In [106]:
import pickle

#serializando o treinamento
cl = NaiveBayesClassifier(clean_data)
with open('class_nb.bin', 'wb') as f:
    pickle.dump(cl, f)